# Brain tumor inference script
## By: Meet Ghodke

A webcam video is active for 10 seconds (in this code, this could also be changed as per user requirements). After the conversion of the transfer learning model from tf to tflite, this tflite model is used to capture and classify each frame from the webcam video. After this, the
inference script is used to allow the inference of model and predict the tumor and confidence in real time.

In [12]:
import tensorflow as tf
import time
import numpy as np
import cv2

## Interpreter

This code will use the tensorflow interpreter for the following functionality:
- Load the Fraunhofer_Image_Classification.tflite model into the code.
- With an image frame taken from the real time video, build a connection between the frame and the .tflite model for the processing of the image.
- The .tflite model classifies the model and depicts the confidence rating, use interpreter to output the results.

In [15]:
interpreter = tf.lite.Interpreter(model_path="/kaggle/input/classification-brain-tumor-tflite-model/classification_brain_tumor.tflite")
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


- `tf.lite.Interpreter()` is used to load the .tflite model into this program.
- `.allocate_tensors()` is a method to allocate memory to the input and output of the model via tensors.
- The input tensor is created to give images captrued via frames from live video as an input to the .tflite model
- The output tensor is to retrieve the image's confidence score and its classification label.

In [16]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
print("Input shape desired:", interpreter.get_input_details()[0]['shape'])

Input shape desired: [  1 224 224   3]


- `.get_input_details()` outputs the input tensor dictionary that has details about the image.
- `.get_output_details()` outputs the predictions in form of a dictionary.
- MobileNetV2 model has only one input and one output. These methods are used to extract a list of tensor descriptions per input/output.
- `[0]` picks the first and only description from that dictionary.
- `['index']` allots a number to the description.

In [17]:
labels = ["Glioma","Meningioma","No tumor","Pituitary"]

In [ ]:
video_capture = cv2.VideoCapture(0)

This command opens the webcam from the pc. .VideoCapture(0) method is used to signify the primary (default) camera of the pc.

In [ ]:
start_seconds = time.time()
total_seconds = 15
while time.time() - start_seconds < total_seconds:
    ret,frame = video_capture.read()
    if ret == True:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(frame,(224,224))
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) 
        image = image.astype(np.float32) / 255.0
        image = np.expand_dims(image,axis=0)
        interpreter.set_tensor(input_index,image)
        interpreter.invoke()
        raw_logits = interpreter.get_tensor(output_index)[0]
        convert_probability = tf.nn.softmax(raw_logits)
        probability_array = convert_probability.numpy()
        label = labels[np.argmax(probability_array)]
        confidence_score = probability_array.max()*100
        font = cv2.FONT_HERSHEY_PLAIN
        font_color = (0,0,0)
        font_size = 2
        font_thickness = 2
        cv2.putText(frame, f"Tumor: {label} , Confidence: {confidence_score:.1f}%", (30, 30), font, font_size, font_color, font_thickness) 
        cv2.putText(frame, "By: Meet Ghodke", (60, 60), font, font_size, font_color, font_thickness)
        cv2.imshow("Inference", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print("No frame")
        break
cv2.imshow("Brain tumor classification", frame)
cv2.waitKey(0)
video_capture.release()
cv2.destroyAllWindows()
        

- `time.time()` starts the timer and records the time that video was captured. It startscounting when the webcam starts recording a video.
- `total_seconds` is the time the user wants to record the video. Here, the user wants torecord this video for 15 seconds.
- `while time.time() - start_seconds < total_seconds` : Run the loop tocapture frames for analysing until the time the user wants the camera to record.(15seconds).
- `ret,frame = video_capture.read()` does the following:
- - `ret` is a variable with boolean data type that returns true if frames from thevideos are available.
- - `frame` is an image vector in BGR format (default colour of OpenCV).
- `if ret == True` indicates that the frame was captured.
- `image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)` converts the RGB format to gray scale (ideal for MRI).
- `image = cv2.resize(frame,(224,224))` resizes the frame to 224*224 dimensionsthat were similar to the default dimensions of MobileNetV2 model.
- `image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)` reverses the gray scale image back to RGB format for MobileNetV2 model input.
- `image = image.astype(np.float32) / 255.0` has two functionalities:
- - `image.astype(np.float32)`: converts image data type to the data type requiredby tflite for inference (32 bit floating point).
- - `/255.0`: converts the values from [0,255] to [0,1] for image classification.
- `image = np.expand_dims(image,axis=0)` adds an extra dimension (number ofbatches). As seen before, the model input desired is [1 , 224 , 224 , 3]
- `interpreter.set_tensor(input_index,image)` sets the image array to the inputtensor input_index in order to run inference.
- `interpreter.invoke()` is the method to run the inference.
- `raw_logits = interpreter.get_tensor(output_index)[0]` generates raw list oflogits from the output tensor.
- `convert_probability = tf.nn.softmax(raw_logits)` converts the raw scoresobtained from output tensor into probabilities that add up to 1.
- `probability_array = convert_probability.numpy()` turns the tensorflowoutput tensor array into a numpy array so that its easier to work with.
- `label = labels[np.argmax(probability_array)]` Returns the highest probability and checks which of the labels provided it corresponds to (0 is Glioma, 1 is Meningioma, 2 is No tumor, 3 is Pituitary)
- `confidence_score = probability_array.max()*100` plugs the highestprobability out of the probability_array and multiplies by 100 giving the confidence score percentage.
- `if cv2.waitKey(1) & 0xFF == ord('q'): break` indicates that if key 'q' ispressed before the `total_seconds`, quit the operation.
- `cv2.waitKey(0)` : When the user presses a key, then close the window.
- `cap.release() and cv2.destroyAllWindows()` is used while closing the opencv library. It indicates to release webcam resources and close all opencv windows.

In [ ]:
print(f"Final Prediction → Tumor: {label}, Confidence: {confidence_score:.1f}%")